### NIFTY 50 

* The NIFTY 50 is a benchmark Indian stock market. It is one of the two main stock indices used in India. 50 represent the 50 largest companies listed on the Nation stock exchange
* In this top 50 we took 3 IT company dataset 
 - wipro
 - TCS
 - HCL_tech

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')

In [ ]:
date_parser = lambda x: pd.datetime.strptime(x, '%Y-%m-%d')

wipro = pd.read_csv('/kaggle/input/nifty50-stock-market-data/WIPRO.csv',  parse_dates = ['Date'], date_parser = date_parser)
TCS = pd.read_csv('/kaggle/input/nifty50-stock-market-data/TCS.csv', parse_dates = ['Date'], date_parser = date_parser)
HCL_tech = pd.read_csv('/kaggle/input/nifty50-stock-market-data/TECHM.csv', parse_dates = ['Date'], date_parser = date_parser)

In [ ]:
wipro.tail(2)

In [ ]:
TCS.head(2)

In [ ]:
HCL_tech.head(2)

## column discription
* Prev Close - Previous day's close price
* Open and close - indicates opening and closing price on a particular day
* low and high - indicates the lowest and height price on a particular day
* VWAP - volume weighted average price
* last - last traded price on a particular day
* Volume - indicates total volume of stocks traded on particular day
* Turnover - indicates total value of stocks traded during a particular period

In [ ]:
print(wipro.shape)
print(TCS.shape)
print(HCL_tech.shape)

In [ ]:
print(wipro.info())
print(TCS.info())
print(HCL_tech.info())

* here we are going to take few months market performance data from 2021 onwards

In [ ]:
wipro = wipro[wipro['Date']>= '2021-01-01']
TCS = TCS[TCS['Date']>= '2021-01-01']
HCL_tech = HCL_tech[HCL_tech['Date']>= '2021-01-01']

In [ ]:
wipro = wipro.set_index('Date')
TCS = TCS.set_index('Date')
HCL_tech = HCL_tech.set_index('Date')

In [ ]:
#The data is stationary
X_wipro = wipro.loc[:,['Trades']]
X_TCS = TCS.loc[:,['Trades']]
X_HCL_tech = HCL_tech.loc[:,['Trades']]

In [ ]:
print(X_wipro.shape)
print(X_TCS.shape)
print(X_HCL_tech.shape)

## Creating an next 1 month dates to prdict the values

In [ ]:
from pandas.tseries.offsets import DateOffset
def future_dates(X):
    future_dates = [X.index[-1]+ DateOffset(days = x) for x in range(0,30)]
    future_dates_df = pd.DataFrame(index=future_dates[1:],columns = X.columns)
    return future_dates_df

In [ ]:
wipro_future_df = pd.concat([X_wipro,future_dates(X_wipro)])
TCS_future_df = pd.concat([X_TCS,future_dates(X_TCS)])
HCL_tech_future_df = pd.concat([X_HCL_tech,future_dates(X_HCL_tech)])

# SARIMAX model

In [ ]:
import statsmodels.api as sm

In [ ]:
def model_train(X, order):
    model=sm.tsa.statespace.SARIMAX(X['Trades'],order=order,seasonal_order=(3,1,4,12))
    results=model.fit()
    return results

### wipro future Trades prediction

In [ ]:
results = model_train(wipro_future_df, order = (3,1,4))
wipro_future_df['forecast'] = results.predict(start = 78 ,dynamic=True)
wipro_future_df[['Trades','forecast']].plot(figsize=(12,8))

### TCS future Trades prediction

In [ ]:
results = model_train(TCS_future_df, order = (2,0,4))
TCS_future_df['forecast'] = results.predict(start = 78,dynamic=True)
TCS_future_df[['Trades','forecast']].plot(figsize=(12,8))

### HCL_tech future Trades prediction

In [ ]:
results = model_train(HCL_tech_future_df, order = (3,0,4))
HCL_tech_future_df['forecast'] = results.predict(start =78,dynamic=True)
HCL_tech_future_df[['Trades','forecast']].plot(figsize=(12,8))

## Thank you..